[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kan-bayashi/INTERSPEECH19_TUTORIAL/blob/master/notebooks/wavenet_vocoder/wavenet_vocoder.ipynb)

# WaveNet Vocoder Recipe Demonstration

**Tomoki Hayashi**

Department of Informatics, Nagoya University  
Human Dataware Lab. Co., Ltd.

Good afternoon, everyone.  
This is Tomoki Hayashi, doctoral researcher @ Nagoya university.  
From here, I will introduce my implementation of wavenet vocoder.

## Access the notebook

https://bit.ly/2kqDCEr

<img src=https://user-images.githubusercontent.com/22779813/64497216-2154e180-d2e7-11e9-861a-7987af223fd0.png width=30%>

If you have not yet open the notebook, please access the notebook from this colab URL.  
You can also access from the my github.

In [ ]:
import time
start_time = time.time()

## Environmental setup

First, install dependecies (It takes several minutes).

First, let us setup the environment.  
Please run the cell and please wait a minute until finish the installation.

In [ ]:
!apt-get install -qq -y bc tree
!git clone https://github.com/kan-bayashi/PytorchWaveNetVocoder.git -b IS19TUTORIAL
!git clone https://github.com/k2kobayashi/sprocket.git -b IS19TUTORIAL
!cd sprocket && pip install -q .
!cd PytorchWaveNetVocoder && pip install -q .
!cd PytorchWaveNetVocoder && mkdir -p tools/venv/bin && touch tools/venv/bin/activate
import sprocket, wavenet_vocoder  # check importable
!echo "Setup done!"

## What is the PytorchWaveNetVocoder?

Github: [kan-bayashi/PytorchWaveNetVocoder](https://github.com/kan-bayashi/PytorchWaveNetVocoder)  
Samples: https://kan-bayashi.github.io/WaveNetVocoderSamples/

- WaveNet vocoder implemention with pytorch
- Support [kaldi](https://github.com/kaldi-asr/kaldi)-like recipes, easy to reproduce the results
- Support [World](https://github.com/mmorise/World) features / mel-spectrogram based models
- Support multi-gpu training / decoding
- Support a noise shaping [[Tachibana+ 2018](https://ieeexplore.ieee.org/document/8461332)]




During the installation, let me introduce my implementation of wavenet vocoder, named pytorch wavenet vocoder.  
You can check the code from my github and listen to the sample in the demo HP.  
This repository provides the wavenet vocoder implementation using pytorch.  
And it supports kaldi-like recipe, easy to reproduce the results.
And support speech analyzer, world-based feature and mel-spectrogram based models.  
So you can use this repository for not only voice conversion but also text-to-speech.  
And it also supports multi-gpu training / decoding, and noise shaping techniques proposed by Tachibana.

## What it the kaldi-like recipe?

Key features:
- Prepared for each corpus (e.g. CMU Arctic, LJSpeech)
- Consists of unified several stages  
  (e.g. data preparation, feature extraction, and so on.)
- Includes all procedures needed to reproduce the results
- All of the recipes are stored in `egs/<corpus>/<type>`.

Maybe the most of you are familiar with kaldi, so here let me introduce kaldi-like recipe briefly.  
Kaldi-like recipes are prepared for each corpus such as CMU Arctic, LJspeech.  
These recipes consist of unified several stages, including data download, preparation, feature extraction, training and so on.  
In other words, each recipe includes all of the procedures which needed to reproduce the results.  
All of the recipes are stored in egs / the name of corpus / the type of recipe.

Supported corpus:
- [CMUArctic database](http://www.festvox.org/cmu_arctic/): `egs/arctic`, 16 kHz, English, Several speakers.
- [LJ Speech database](https://keithito.com/LJ-Speech-Dataset/): `egs/ljspeech` 22.05 kHz, English, Single female speaker.
- [M-AILABS speech database](http://www.m-ailabs.bayern/en/the-mailabs-speech-dataset/):`egs/m-ailabs-speech`: 16 kHz, various speakers

About supported type, see detail in https://github.com/kan-bayashi/PytorchWaveNetVocoder/tree/master/egs

Here is the list of supported corpus.
CMU Arctic, LJSpeech, M-AILABS Speech dataset.
About the supported recipe type, please check the URL.

Of course you can add your own recipe using your own corpus.  
After this demonstration, maybe you can understand how to apply your dataset to make recipe.

## Run the demo recipe

Let us run the demo recipe `egs/arctic/sd-mini`.

- Small version of `egs/arctic/sd`
- Use subset of all of the utterances
- **Cannot build a good model** but the flow is **the same**

You can understand each stage within 30 minutes!

OK. Maybe you finished the installation.  
Let us start the running the recipe.  
In this demonstration, we use egs/arctic/sd-mini recipe, which is a small version of sd reicpe.  
It uses subset of all of the utterances and unfortunately cannot build a good model but the flow is the completely same.  
You can understand each stage within 30 minutes.  
Let move on the recipe directory. Please run the cell.

In [ ]:
# move on the recipe directory
import os
os.chdir("./PytorchWaveNetVocoder/egs/arctic/sd-mini")
!echo $(pwd)

Files in the recipe are as follows:
- `conf`: Directory including config files
- `path.sh`: Script to set the environmental variables.
- `run.sh`: Main script.

In [ ]:
!tree -L 1

First, I will introduce the files included in the recipe directory.  
Let me check the files.  
Conf is the directory including config files, path.sh is the script ot set the environmental variable such as CUDA, and run.sh is the main script.

`conf` includes f0 setting files whose name format is `<speaker_name>.f0`.

In [ ]:
!ls conf

Let me check the files in the conf directory.  
conf includes f0 setting files whose name format is speaker_name.f0.

`<speaker_name>.f0` includes `min_f0 max_f0`.  
These values are predecided by ourselve, so you can modify them.

In [ ]:
!cat conf/slt.f0  # (minf0 maxf0)

conf includes f0 setting files whose name format is <speaker_name>.f0.  
These files are used for world feature extraction.  
And each <speaker_name>.f0 includes min_f0 and max_f0 values.  
These values are predecided by ourselve, so you can modify them.

All of the hyperparameters are written in `run.sh`.

In [ ]:
!head -n 69 run.sh 

Let us introduce these parameters in detail later.

All of the hyperparameters are written in the main script run.sh.
Let me check the all of the hyperparameters.  
These parameters can be changed from command line. of course you can directly edit the file.  
Each of hyperparameters is explained in detail later.

In [ ]:
# (Optional) here you can add your command to check the file!


### Overview of the recipe

<div align="center">
    <img src=figs/overview.png width=80%>
</div>

OK. Let us move ont the introduction of the flow of the recipe.  
This figure is the overview of recipe.  
The recipe consists of 7 stages from 0 to 6.  
In stage 0, the corpus will be donwloaded from cloud and then split the corpus into training and evaluation set.  
Stage 1 performs preprocessing and feature extraction.  
Stage 2 calculates the statistics of extracted features.  
Stage 3 performs noise weighting for training waveforms.  
Stage 4 trains WaveNet using extracted features and noise weighted waveforms.  
Stage 5 decodes waveform of evaluation data using trained WaveNet.  
And finaly, Stage 6 perform noise shaping for generated waveforms.

If run `run.sh`, all of stages will be performed.

But we can specify the stage to run with `--stage` options.

- `run.sh --stage 0`: Run only the stage 0
- `run.sh --stage 012`: Run the stages 0, 1, and 2.

Here, let us run each stage step-by-step.

If you run the main script run.sh, all of the above stages will be performed.  
But we can specify the stage to be running with --stage option like this.  
Here, to understand each stage, let us run each stage step-by-step.

### Stage 0: Data preparation

This stage performs download of corpus and list preparation.

<div align="center">
    <img src=figs/stage_0.png width=70%>
</div>

First, stage 0, data prepration.  
This stage downloads corpus from cloud and make the list of utterances.

In arctic, there are seven speakers.  
Here let us use `slt` to build a model.

In [ ]:
# you can specify the speaker via --spk (default=slt)
!./run.sh --stage 0 --spk slt

In arctic, seven speakers exists.  
Here let us use female speaker slt to build a model.  
You can specify the speaker with `--spk` option.  
Please run the cell.

Corpus is saved in
- `downloads/cmu_us_<spk_name>_arctic_mini`

Two lists of wav files are created.
- `data/tr_slt/wav.scp`: wav list file for training
- `data/ev_slt/wav.scp`: wav list file for evaluation

In [ ]:
!tree -L 3 -I local

Let me check the file. Please run the cell.  
The downloaded corpus is saved in downloads/cmu_us_.  
And 2 lists of wav files are created under the data directory.  
data/tr_slt/wav.scp is for training and the other is for evaluation.

The list file is that:
- Each line has the path of wav file
- All of the lines are sorted

In [ ]:
 !head -n 3 data/*_slt/wav.scp

Here we use 32 utts for training, 4 for evaluation.

In [ ]:
!wc -l < data/tr_slt/wav.scp
!wc -l < data/ev_slt/wav.scp

Let me check the list file format. Please run the cell.  
You can see that each line has the path of wav file and all of the lines are sorted.  
Let me check the number of lines via wc commnd.  
Here we use 32 utterances for training and 4 utterances for evalaution.

In [ ]:
# (Optional) here you can check the file with your commands!


### Stage 1: Feature extraction

This stage performs feature extraction with the
list file.

<div align="center">
    <img src=figs/stage_1.png width=70%>
</div>

Next, stage 1 feature extration.  
This stage applies high pass filter as preprocessing and then extract feature vector using world analyzer.

In [ ]:
# Hyperparameters related to stage 1
!head -n 36 run.sh | tail -n 13

At first, let me check the hyperparameters related to feature extraction.

In [ ]:
# run stage 1 with default settings
!./run.sh --stage 1

Please run the stage 1.  
In the log, they show the hyperparameters and progress.  
And this stage is performed in parallel using the multiprocessing.

Hyperparameters can be changed via command line.  
But it will overwrite the existing ones. Be careful.

In [ ]:
# example of changing hyperparameters of feature extraction
# !./run.sh --stage 1 --mcep_dim 30 --shiftms 10 

If you want to change the hyperparameters of feature extraction, we can change by specifiying the option from command line.  
But the feature vector will be overwritten, please be careful.

Extracted features are saved as `hdf5` in
- `hdf5/tr_slt/*.h5`: Feature file of training data 
- `hdf5/ev_slt/*.h5`: Feature file of evaluation data

Lists of feature files are created 
- `data/tr_slt/feats.scp`
- `data/ev_slt/feats.scp`

High pass filtered training wav files are saved in
- `wav_hpf/tr_slt/*.wav`: Filtered wav file of training data

List of filetered wav files is created
- `data/tr_slt/wav_hpf.scp`: List of filtered wav files

In [ ]:
!tree -L 3 -I "*.f0|local|cmu_*"

OK. Let me check the created files.  
Extracted features are saved in the hdf5 format under hdf5 directory.  
hdf5/tr_slt includes feature files for training and hdf5/ev_lst includes oens for evaluation.  
Also lists of these feature vector are created under data directory.
data/tr_slt/feats.scp and data/ev_slt/feats.scp.

And for training data, high pass filetered waveform is saved uder wav_hpf directory.  
The list of filtered wav files is also created. 

Let us check the list file format:
- Each line has the path of feature or wav
file  
- All of the lines are sorted
- Assume that all of the lists has the same
order

In [ ]:
!head -n 3 data/*_slt/feats.scp
!echo ""
!head -n 3 data/tr_slt/wav_hpf.scp

Let me check the format of list files.  
As the same as wav.scp files, each line has the path of feature or wav file, and all of the lines are sorted.  
We assume that all of the lists has the same order.

hdf5 format can be loaded as `numpy.ndarray` in python using `h5py` library.

In [ ]:
import h5py
with h5py.File("hdf5/tr_slt/arctic_a0001.h5") as f:
    print(f.keys())
    feat = f["world"][()]
# or you can use our utils
from wavenet_vocoder.utils import read_hdf5
feat = read_hdf5("hdf5/tr_slt/arctic_a0001.h5", "world")
print("Feature shape: (#num_frames=%d, #feature_dims=%d)" % (feat.shape[0], feat.shape[1]))

The extracted hdf5 file can be loaded as numpy.ndarray using h5py.   
Here we load the feature via h5py.  
Also we provide the utilities tool read_hdf5.  
You can use this instead of h5py library.

The feature is extracted with World.
- `U/V binary` (1 dim)
- `continuous F0` (1 dim), 
- `mcep`(25 dim = `mcep_dim + 1`) 
- `ap` (1 dim).

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))
plt.subplot(2, 2, 1)
plt.plot(feat[:, 0])
plt.title("U/V binary")
plt.subplot(2, 2, 3)
plt.plot(feat[:, 1])
plt.title("Continuous F0")
plt.subplot(2, 2, 2)
plt.imshow(feat[:, 2:26].T, aspect="auto")
plt.title("Mel-cepstrum")
plt.subplot(2, 2, 4)
plt.plot(feat[:, -1])
plt.title("Aperiodicity")
plt.tight_layout()
plt.show()

The dimenstion of feature vector is 28.  
Let me check each components of feature vector.  
The feature is extracted with World analyzer.  
Let me visualize each features.
U/V binary, contious F0, mel-ceptrum, and aperiodicity components.

In [ ]:
# (Optional) here you can check the file with your commands!


### Stage 2: Statistics calculation

This stage calculates the mean and variance of features.

<div align="center">
    <img src=figs/stage_2.png width=70%>
</div>

Next, stage 2 statistics calculation.  
This stage calculate the mean and variacne of features for feature normalization and filtering.

In [ ]:
# run stage 2 with default settings
!./run.sh --stage 2

Please run the stage 2.

Calculated statistics are saved as `hdf5` format in
- `data/tr_slt/stats.h5`

`stats.h5` is used for:
- Feature normalization during training
- Calculation of noise shaping filter coefficient

In [ ]:
!tree -L 3 -I "*.f0|*.wav|*[0-9].h5|local|cmu_*"

Through the stage 2, statistics file is saved as hdf5 format under the data/tr_slt.  
Let me check the files.  
`stats.h5` is the statistics of the training data.  
`stats.h5` is used for feature normalization during training and calculation of noise shaping coefficient.  

`stats.h5` can be loaded as follows:

In [ ]:
with h5py.File("data/tr_slt/stats.h5") as f:
    print(f.keys())
    print(f['world'].keys())
    mean = f['world']['mean'][()]
    scale = f['world']['scale'][()]
    print(mean.shape)
    print(scale.shape)
    
# or you use our utils
mean = read_hdf5("data/tr_slt/stats.h5", "world/mean")
scale = read_hdf5("data/tr_slt/stats.h5", "world/scale")

Let me load the statistics file.  
It contains mean and scale, which is a variance.

In [ ]:
# here you can check the file with your commands!


### Stage 3: Noise weighting

This stage applies noise weighting filter to training
wav files.

<div align="center">
    <img src=figs/stage_3.png width=70%>
</div>

Next, stage 3, noise weighting.  
This stage applies noise weighting filter to high pass filtered wav files.

In [ ]:
# Hyperparameters related to stage 3
!head -n 38 run.sh | tail -n 2

Let me check the hyperparameters related to stage 3.  
The hyperparametres are onlly use_noise_shaping which decides whether to perform noise shapng, 
and mag, which is the strength value from 0.0 to 1.0.  
Experimentaly around 0.5 is a good for the best quality.  
You can see the detail investigation in Dr. Tachibana's paper.

In [ ]:
# run stage 3 with default settings
!./run.sh --stage 3

If `use_noise_shaping=false`, `stage 3` will be skipped.

Please run the stage 3.  
Note that if you specify use_noise_shaping = false, this stage will be skipped.

Noise weighting filtered wav files are saved in
- `wav_nwf/tr_slt/*.wav`

The list of noise weighting filtered wav files is saved as
- `data/tr_slt/wav_nwf.scp`

In [ ]:
!tree -L 3 -I "*.f0|*[0-9].h5|local|cmu_*"

Let me chech the created files.   
Noise weighting filtered wav files are saved in wav_nwf/tr_slt.  
The list of noise weighting filtered wav files is saved as data/tr_slt/wav_nwf.scp  

Let us check the difference of waveform here.

In [ ]:
# listen to the samples
import IPython.display
IPython.display.display(IPython.display.Audio("wav_hpf/tr_slt/arctic_a0001.wav"))
IPython.display.display(IPython.display.Audio("wav_nwf/tr_slt/arctic_a0001.wav"))

In [ ]:
# show spectrogram
import soundfile as sf
import matplotlib.pyplot as plt
x, fs = sf.read("wav_hpf/tr_slt/arctic_a0001.wav")
x_ns, fs = sf.read("wav_nwf/tr_slt/arctic_a0001.wav")
plt.figure(figsize=(16, 7))
plt.subplot(1, 2, 1)
plt.specgram(x, Fs=fs)
plt.title("Original spectrogram")
plt.subplot(1, 2, 2)
plt.specgram(x_ns, Fs=fs)
plt.title("Noise weighting filtered spectrogram")  

Here, let me check the difference.  
Please run the cell.  
Maybe you can understand the difference between them.  
Let me also check the spectrogram. Please run the cell.
You can confirm that the power in highfreqency became bigger and that in low frequency did smaller.

Filtering related parameters `mlas/coef` and `mlsa/alpha` are added in `data/tr_slt/stats.h5`.

In [ ]:
with h5py.File("data/tr_slt/stats.h5") as f:
    print(f.keys())
    print(f["mlsa"].keys())
    print(f["mlsa"]["alpha"])
    print(f["mlsa"]["coef"])

`mlsa/coef` is the coefficient of MLSA filter, which is calculated from averaged mel-cepstrum and `mag`.  
`mlsa/alpha` is the hyperparameter `alpha`, all pass filter coefficient.

And through this stage, additional parameters are written in the statistics file `stats.h5`.  
Let me check added parameters.  
One is the mlsa/coef, which is the coefficient of MLSA fileter.  
They are calculated from averated mel-ceptrum and mag parameters, as you saw in the Dr. Toda's lecture.  
The other is the mlsa/alpha, which the all pass filter coefficient.  
These parameters are used in the final stage, noise shaping.

In [ ]:
# (Optional) here you can check the file with your commands!


### Stage 4: WaveNet training

This stage trains WaveNet using extracted
features and noise weighting filtered wav files.

<div align="center">
    <img src=figs/stage_4.png width=70%>
</div>

I'm sorry I kept you waiting.  Now ready to train the network!  
Next is the stage 4, WaveNet training.

In [ ]:
# Hyperparameters related to stage 4
!head -n 59 run.sh | tail -n 19

First, please run the cell to check the hyperparameters.  
There are several parameters related to network structure, and the others are realted to training.  
I will explain in later.

In [ ]:
# run stage 4 with default settings
!./run.sh --stage 4 --iters 500

Please run the stage 4.  Here we train 500 iteration by specifying the hyperparameters iters=500.

Default network structure in `egs/arctic/sd-mini`.
<div align="center">
    <img src=figs/wavenet.png width=70%>
</div>

During the training, I will explain the relationship between the hyperparameters and network structure.  
This figure shows the wavenet structure.  
The hyperparameters related to network structure are 
- n_aux, which is the number of auxiliary features
- n_resch. which is the number of channel in convoluional layer placed in residual blocks,
- n_skipch, which is the number of channles in convolutional layer connected to output layers.
- n_quantize, which is the number of classes of discretized wavenform,
In my implementation, we use one-hot vector formated waveform as a inputs.

Maybe you can understand the relationship between the hyperparameters and the structure.

Example when `dilation_depth=3` and `dilation_repeat=2`.

<div align="center">
    <img src=figs/structure_ex.png width=45%>
</div>

Remaining hyperparameters related to the structure are dilation_depth and dilation_repeat.  
I will explain these parameters using the example.  
This figure shows the example of diation_depth=3 and dilation_repeat=2.  
The number of residual blocks is equal to dilation depth times dilation repeat, in this case the number of blocks is 6.  
Each of dilation in each residual block starts from 2 to the power of 0, 2 to the power of 1, 2 to the power 2 and then repeat them.

Make a batch by split a waveform into pieces.
- `batch_size`: Number of batches
- `batch_length`: Length of each batch

<div align="center">
    <img src=figs/batch.png width=65%>
</div>

Next, I will explain how to make the batch.
This figure shows the overview of batch making.    
The batch will be created in the windowing manner.
batch_length decides the length of each piece and batch_size decides the number of pieces.  
The length of each piece is batch_length + receptive field size and the next batch will be created with receptive_filed size overlap.  
This is because we do not calculate the loss for corresponding to within receptive fields.  
OK. Maybe you finished the training. Let me check the results.

Model parameters are saved as  
- `exp/tr_arctic_16k_sd_world_slt_*/checkpoint-*.pkl` 

Modle configuration is saved as  
- `exp/tr_arctic_16k_sd_world_slt_*/model.conf`

The directory name is automatically set to be unique depending on hyperparameters.

In [ ]:
!tree -L 3 -I "*.f0|*.wav|*[0-9].h5|*.scp|*.log|local|cmu_*"

Let us check the files. please run the cell.  
Model parameters are saved in exp/tr_buraburabura/checkpoint.pkl  and model configuration file is saved as model.conf in the same directory.  
The name of the directory is automatically set o be unique depending on hyperparameters.  
So if you change the hyperparemeters, the model will be saved in the different directory,  
You do not need to care about the overwriting.  
The log is also saved in log directory in the same directory.

Model configuration file can be loaded as `argparse.Namespace`.

In [ ]:
import torch
conf = torch.load("exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up/model.conf")
print(conf)

Here, let me load the model.conf in python.  
This file can be loaded using pytorch load function and then loaded as Namespace like this.  
It contains the hyperparemters to build the network.

Model parameters `checkpoint-*.pkl` can be loaded as `dict` which contains
following information:
- `iterations`: Number of iterations of this parameters
- `optimizer`: `Dict` of states of optimizer
- `model`: `OrderedDict` of Model
parameters

In [ ]:
state_dict = torch.load("exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up/checkpoint-500.pkl")
print(state_dict.keys())
print(state_dict["iterations"])
print(state_dict["optimizer"].keys())
print(state_dict["model"].keys())

Let me load the model parameter file checkpoint.pkl. Please run the cell.  
This file contains the number of iteration, optimizer states, and model parameters.  

You can resume training from `checkpoint-*.pkl` file with `--resume` options.

In [ ]:
!./run.sh --stage 4 \
    --iters 1000 \
    --resume exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up/checkpoint-500.pkl

So you can resume the training from checkpoint file.  
Here, let us resume the training from 500 iters checkpoint and then continue to train until 1000 iters.

You can train using multi-gpu with `--n_gpus` option.

In [ ]:
# In colab, we can use only a single gpu :(
# batch_size must be >= n_gpus
# !./run.sh --stage 4 --n_gpus 2 --batch_size 2

Also, if you want to use multi gpu, please use n_gpus options.  
Note that please set batch_size to be greater than or equal to n_gpus in the case of multi gpus.  
Unfortunately, we cannot use multi gpu so skipped it.

In [ ]:
# here you can check the file with your commands!


### Stage 5: WaveNet decoding

This stage performs decoding of evaluation data.

<div align="center">
    <img src=figs/stage_5.png width=70%>
</div>

Next, stage 5, WaveNet training.  
This stage performs decoding using evaluation set features.

In [ ]:
# Hyperparameters related to stage 5
!head -n 69 run.sh | tail -n 9

Let me check the hyperparameters related to stage 5.  
Please run the cell.  

(After running training)  
There are some hyperparameters to specifying the model to be used.  
If you not provide these hyperparameters, the script will use checkpoint-final.pkl corresponding to specfied hyperparameters.  
So the important one is the decode_batch_size.  
In this implementation, decoding is performed in batch mode, it can reduce the generation time.  
But it requires GPU memory. So if you want to generated very long utterances, please be careful to avoid out of memory error.

In [ ]:
# run stage 5 with default setting
!./run.sh --stage 5

Before the explanation of hyperparameters, let us run the stage 5.  
It takes time. please keep waiting.

You can specify the `checkpoint-*.pkl` file used for decoding and directory to
be saved via `--checkpoint` and `--outdir` options.

In [ ]:
# it takes times, comment out
# !./run.sh --stage 5 \
#     --checkpoint exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up/checkpoint-100.pkl
#     --outdir exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up/wav_ckpt_100

For example, you can specify the model parameter file to be used for decoding like this.  
It takes time, we skip it.  

We can use multi-gpu decoding via `--n_gpus` option.

In [ ]:
# In colab, we can use only a single gpu :(
# !./run.sh --stage 5 --n_gpus 2

Also, we can perform multi-gpu decoding.  you can use --n_gpus options like this but does not work in colab.

### MOS on Naturalness

<img src=https://camo.githubusercontent.com/b1b752447f3d040901fae34fefca03fc0765d187/68747470733a2f2f6b616e2d626179617368692e6769746875622e696f2f576176654e6574566f636f64657253616d706c65732f696d616765732f6d6f735f6e756d5f747261696e2e626d70 width=70%>

This figure shows the result of MOS on naturalness.  
Vertical axis represents MOS on naturalness and horizon one represents the type of model.  
The number in the brackets represents the number of utterances of training data.  
From this figure, we can build a good sd wavenet vocoder if we parepare greater than 300 uttrances. 

Generated wav files are saved in 
- `exp/tr_arctic_sd_tr_arctic_16k_sd_*/wav`

In [ ]:
!tree exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up

OK. I hope we finsh the decoding.  
Let me check the generated wav files. please run the cell.  
Generated wav files are saved in the wav directory located in the model directory.  
These wav files are based on the noise weighting filtered one, so we need to restore to original waveform scale.  

In [ ]:
# (Optional) here you can check the file with your commands!


### Stage 6: Noise shaping

This stage applies noise shaping filter to generated wav files.

<div align="center">
    <img src=figs/stage_6.png width=70%>
</div>

OK. The next stage is the final.  
Stage 6, noise shaping.  
This stage applies noise shaping filter, which is the inverse filter of noise weighting filter, to generated wav files to restore them into original scale.

In [ ]:
# run stage 6 with default setting
!./run.sh --stage 6

Please run the stage 6.

Restored wav files are saved in

- `exp/tr_arctic_sd_tr_arctic_16k_sd_*/wav_nsf`

In [ ]:
!tree exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up

Restored wav fiels are saved in wav_nsf directory located in the model directory.  
Let me check the files.

In [ ]:
# (Optional) here you can check the file with your commands!


Finished! Unfortunately, generated samples are just-like a noise.  
So Let us check the samples which trained with `egs/arctic/sd` from  
https://kan-bayashi.github.io/WaveNetVocoderSamples/

## Use pretrained model as vocoder

Here we show how-to-use pretrained model as
vocoder.  
What we need to prepare is following three files:

- `model.conf`:
Model configuration file.
- `checkpoint-*.pkl`: Model parameter file.
- `stats.h5`: Statistics file.

Let us pack following files into
`pretrained_model/` directory.

In [ ]:
# summarize trained model in the directory
!mkdir pretrained_model
!cp -v exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up/stats.h5 \
    exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up/model.conf \
    exp/tr_arctic_16k_sd_world_slt_nq256_na28_nrc32_nsc16_ks2_dp5_dr1_lr1e-4_wd0.0_bl10000_bs1_ns_up/checkpoint-1000.pkl \
    pretrained_model

First, please prepare the list file of feature files to be decoded.

In [ ]:
import os
import numpy as np

# here make a dummy features and the stored as hdf5 with key "/world"
os.makedirs("dummy", exist_ok=True)
for idx, n_frames in enumerate([10, 20, 30, 40]): 
    x = np.random.randn(n_frames, 28)  # (#num_frames, #feature_dims)
    with h5py.File("dummy/dummy_%d.h5" % idx, "w") as f:
        f["world"] = x

# make a list of features to be decoded.
!find dummy -name "*.h5" > dummy_feats.scp

# check
!cat dummy_feats.scp

Run the `--stage 56` by specifying `--feats` in the recipe directory.

In [ ]:
# decode with pretrained model through the recipe
!./run.sh --stage 56 \
    --outdir dummy_feats_wav \
    --feats dummy_feats.scp \
    --checkpoint pretrained_model/checkpoint-1000.pkl
!ls dummy_feats_wav*

If you want to use outside of the recipe, directly call python scripts stored in
`wavenet_vocoder/bin`.

In [ ]:
# decode with pretrained model
!python ../../../wavenet_vocoder/bin/decode.py \
     --feats dummy_feats.scp \
     --outdir dummy_feats_wav_2 \
     --checkpoint pretrained_model/checkpoint-1000.pkl \
     --fs 16000 \
     --n_gpus 1 \
     --batch_size 4
# make list of wav files to be filtered
!find dummy_feats_wav_2 -name "*.wav" > dummy_feats_wav_2/wav.scp
# apply noise shaping filter
!python ../../../wavenet_vocoder/bin/noise_shaping.py \
     --waveforms dummy_feats_wav_2/wav.scp \
     --outdir dummy_feats_wav_2_nsf \
     --stats pretrained_model/stats.h5 \
     --fs 16000 \
     --shiftms 5
!ls dummy_feats_wav_2*

## Combine with Sprocket

Let us show how-to-combine wavenet vocoder with voice conversion toolkit [sprocket](https://github.com/k2kobayashi/sprocket).    
Here, we generate converted voice with pretrained models.

Oh time is limited, we skip the usage of pretrained model part.  
Let's move on the final part, combine with sprocket.  
This part is corresponding to the final recipe 3, explain Dr. Kobayashi slides.

In this part, we show how to combine wavenet vocoder with voice conversion toolkit sprocket.  
We try to generated voice with pretrained models.

In [ ]:
# changed directory
!mkdir ../../../../conversion_example
os.chdir("../../../../conversion_example")
!pwd

First, let us move on the directory.

First, download pretrained models.

In [ ]:
# download sprocket model
!../PytorchWaveNetVocoder/wavenet_vocoder/utils/download_from_google_drive.sh \
    "https://drive.google.com/open?id=1PiGDyYDQt0b4h6KAV1MOmDxHjHUv1cT6" \
    downloads/sprocket_pretrained

# download wavenet vocoder model
!../PytorchWaveNetVocoder/wavenet_vocoder/utils/download_from_google_drive.sh \
    "https://drive.google.com/open?id=1AhtRB0vTkjDrum-dfgaiXnQgsAAiYMGW" \
    downloads/wavenet_vocoder_pretrained

# download wav samples
!../PytorchWaveNetVocoder/wavenet_vocoder/utils/download_from_google_drive.sh \
    "https://drive.google.com/open?id=1kBwF7ejyCR5aI9FitmMSCnWdPCNVouqg"

Here let us download sprocket pretrained model, and wavenet vocoder pretrained model, and sample speech for the inputs.

- Sprocket pretrained model
    - `GMM_mcep.pkl`: GMM param file for mcep conversion.
    - `<src_spk>.yml`: Source speaker yaml file.
    - `<src_spk>-<tar_spk>.yml`: Source-target speaker pair yaml file.
    - `<src_spk>.h5`: Statistics file of source speaker.
    - `<tar_spk>.h5`: Statistics file of target speaker.
    - `cvgv.h5`: Statistics file of global variance for converted features.
    
- Target speaker WaveNet vocoder pretrained model
    - `model.conf`: Model configuration file.
    - `checkpoint-*.pkl`: Model parameter file.
    - `stats.h5`: Statistics file.

In [ ]:
!ls downloads/*pretrained

Let me check the downloaded pretrained models.  
Sprocket model includes ...  
And wavenet vocoder model includes ...

Next, extract features and then convert them to target speaker's one.  

In [ ]:
![ ! -e hdf5 ] && mkdir hdf5
![ ! -e wav ] && mkdir wav
!PYTHONPATH=../sprocket/example/src \
    python ../sprocket/sprocket/bin/convert_feats.py \
        --cvmcep0th True \
        --cvcodeap True \
        --cvgvstats downloads/sprocket_pretrained/cvgv.h5 \
        --org_yml downloads/sprocket_pretrained/rms.yml \
        --pair_yml downloads/sprocket_pretrained/rms-slt.yml \
        --org_stats downloads/sprocket_pretrained/rms.h5 \
        --tar_stats downloads/sprocket_pretrained/slt.h5 \
        --mcepgmmf downloads/sprocket_pretrained/GMM_mcep.pkl \
        --iwav downloads/samples/src/arctic_b0536.wav \
        --cvfeats hdf5/arctic_b0536.h5 \
        --owav wav/arctic_b0536.wav
!ls hdf5 wav

Next, extract features and then convert them to target speaker's one.  
Input waveform is specified by `--iwav` and then converted feats are saved as `--cvfeats`, vocoded waveform is saved as `--owav`.
Converted feature vector is saved in hdf5 directory, and converted voice with vocoder is saved in wav.

Then generate waveform with pretrained wavenet using converted features.

In [ ]:
# NOTE: require too much time.
# decode with wavenet vocoder
!find hdf5 -name "*.h5" > hdf5/feats.scp
!python ../PytorchWaveNetVocoder/wavenet_vocoder/bin/decode.py \
     --feats hdf5/feats.scp \
     --outdir wav_wnv \
     --checkpoint downloads/wavenet_vocoder_pretrained/checkpoint-final.pkl \
     --fs 16000 \
     --n_gpus 1 \
     --batch_size 4
# apply noise shaping filter
!find wav_wnv -name "*.wav" > wav_wnv/wav.scp
!python ../PytorchWaveNetVocoder/wavenet_vocoder/bin/noise_shaping.py \
     --waveforms wav_wnv/wav.scp \
     --outdir wav_wnv_nsf \
     --stats downloads/wavenet_vocoder_pretrained/stats.h5 \
     --fs 16000 \
     --shiftms 5

And then let us generate with wavenet vocoder!  
You can run this cell but it takes time.  
So we have put pre-generated ones in `samples`.  
Let us listen to them.

In [ ]:
# listen to pre-synthesized ones
import IPython.display
print("Source")
IPython.display.display(IPython.display.Audio("downloads/samples/src/arctic_b0536.wav"))
print("Target")
IPython.display.display(IPython.display.Audio("downloads/samples/tar/arctic_b0536.wav"))
print("Converted voice with vocoder")
IPython.display.display(IPython.display.Audio("downloads/samples/vocoder/arctic_b0536.wav"))
print("Converted voice with wavenet vocoder")
IPython.display.display(IPython.display.Audio("downloads/samples/wavenet_vocoder/arctic_b0536.wav"))

I will play source voice, target  voice, converted voice with vocoder, and that with wavenet vocoder.  
Maybe you can understand the difference with headphone.

In [ ]:
print("running time = %s minite" % ((time.time() - start_time) / 60))

## Conclusion

- Introduced voice conversion with direct waveform modeling
- Introduced Sprocket /  PytorchWaveNetVocoder
    - Can build GMM-based VC / DIFFVC  & WaveNet vocoder
    - Can combine both module to generate high quality converted voices

Thank you for your attendance!  
If you have time, please send us feedback via [Google form](https://forms.gle/28QrvGRBAAiKpWas8). 

That's it. I conclude our tutorial.  
In this tutorial, we introduced voice conversion with direct waveform modeling and introduce sprocket and pytorch wavenet vocoder.  
I hope that now you are ready to start the research of voice conversion!  
Thank you for you attendance! if you have time please send us feedback via google form!